In [1]:
import os
%pwd

'd:\\projectFiles\\text_summarization_project\\research'

In [2]:
os.chdir("../")


In [3]:
%pwd

'd:\\projectFiles\\text_summarization_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import torch

In [5]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [7]:
CONFIG_FILE_PATH = "D:\\projectFiles\\text_summarization_project\\config\\config.yaml"
PARAMS_FILE_PATH = "D:\\projectFiles\\text_summarization_project\\params.yaml"

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        # Convert the file paths to pathlib.Path objects
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)

        # Print the paths when initializing
        print("Config File Path:", config_filepath)
        print("Params File Path:", params_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        root_dir = Path(config.root_dir)

        create_directories([root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.evaluation_strategy,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Use the specified model checkpoint
        model_checkpoint = "google/pegasus-xsum"
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_tf=True).to(device)

        # Initialize the tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading data
        dataset_samsum_pt = load_from_disk(str(self.config.data_path))

        dataset_samsum_pt = dataset_samsum_pt.map(lambda example: {key: example[key][:10] for key in example})

        # Set the output directory to the desired path
        output_dir = "D:\\projectFiles\\text_summarization_project\\model_data"

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=50,
            per_device_train_batch_size=1,  # Reduce this value
            per_device_eval_batch_size=1,   # Reduce this value
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=50,
            save_steps=1e6,
            gradient_accumulation_steps=16
        )


        trainer = Trainer(
            model=model_pegasus, args=trainer_args,
            tokenizer=tokenizer, data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        # Save model to the specified directory
        model_pegasus.save_pretrained(output_dir)
        # Save tokenizer to the specified directory
        tokenizer.save_pretrained(output_dir)

# Assuming the rest of the code for configuration is the same as before



In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

Config File Path: D:\projectFiles\text_summarization_project\config\config.yaml
Params File Path: D:\projectFiles\text_summarization_project\params.yaml
[2024-01-01 18:12:40,513: INFO: common:yaml file: D:\projectFiles\text_summarization_project\config\config.yaml loaded successfully]


[2024-01-01 18:12:40,550: INFO: common:yaml file: D:\projectFiles\text_summarization_project\params.yaml loaded successfully]
[2024-01-01 18:12:40,559: INFO: common:created directory at: artifacts]
[2024-01-01 18:12:40,560: INFO: common:created directory at: artifacts\model_trainer]


ResourceExhaustedError: {{function_node __wrapped__AssignVariableOp_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[96103,1024] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:AssignVariableOp] name: 